In [12]:
import pandas as pd

filename = "results/test-extrinsic-100"
# filename = "results/test-xent-100"

df_aggregated = pd.read_csv(f"{filename}.csv").sort_values("factual", ascending=False)
df_sums = pd.read_json(f"{filename}-summaries.json")

df_aggregated

,model,factual,non_factual,non_factual_extrinsic,non_factual_intrinsic,unknown,failed,ents_per_sum,sum_with_extrinsic,rouge1,rouge2,rougeL
1,fbs_classifier,0.52,0.48,0.29,0.07,0.00,14,3.383721,0.76,0.463955,0.226476,0.362109
2,fbs_classifier_i10,0.52,0.48,0.24,0.06,0.00,19,3.358025,0.71,0.462410,0.229059,0.363073
0,fbs_oracle,0.50,0.49,0.39,0.10,0.01,3,3.628866,0.89,0.471261,0.235422,0.371902
4,corrector,0.46,0.53,0.47,0.08,0.01,0,3.660000,0.92,0.466220,0.225790,0.371539
3,baseline,0.42,0.57,0.51,0.09,0.01,0,3.690000,0.95,0.471603,0.233643,0.373804


In [3]:
df_sums.head()

,fbs_oracle_summary,fbs_oracle_is_non_factual,fbs_oracle_is_factual,fbs_oracle_has_unknown,fbs_oracle_has_failed,fbs_oracle_n_extrinsic,fbs_classifier_summary,fbs_classifier_is_non_factual,fbs_classifier_is_factual,fbs_classifier_has_unknown,...,baseline_is_factual,baseline_has_unknown,baseline_has_failed,baseline_n_extrinsic,corrector_summary,corrector_is_non_factual,corrector_is_factual,corrector_has_unknown,corrector_has_failed,corrector_n_extrinsic
17996567,Plans to cut home-to-school transport subsidie...,True,False,False,False,1,Plans to cut home-to-school transport subsidie...,True,False,False,...,False,False,False,1,Plans to cut home-to-school transport subsidie...,True,False,False,False,1
26398483,Police have released an E-fit image of a man t...,False,True,False,False,1,<Failed generation: blocked all beams>,True,False,False,...,False,False,False,1,Police have released an image of a man they wa...,True,False,False,False,1
30657223,Hartlepool United have signed Reading midfield...,True,False,False,False,4,<Failed generation: blocked all beams>,True,False,False,...,False,False,False,5,Hartlepool United have signed Stags midfielder...,True,False,False,False,5
31113635,The amount of money spent on film production i...,False,True,False,False,2,The amount of money spent on film production i...,False,True,False,...,True,False,False,2,The amount of money spent on film production i...,False,True,False,False,2
32557344,The RSPCA has appealed for information after a...,False,True,False,False,1,The RSPCA has appealed for information after a...,False,True,False,...,True,False,False,1,The RSPCA has appealed for information after a...,False,True,False,False,1


In [4]:
# Helper method
def print_summaries(df, models):
    for sum_id in df.index:
        print(f"---XSUM ID {sum_id}---")
        row = df.loc[sum_id]
        for model in models:
            n_ents = row[model + '_n_extrinsic']
            summary = row[model + '_summary']
            print(f"{model} ({n_ents} extrinsic ents): {[summary]}")
        print()

### Summaries corrected by Oracle

In [5]:
df_inspect = df_sums[
    df_sums.fbs_oracle_is_factual
    & df_sums.baseline_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_oracle"
    ]
)

8
---XSUM ID 26398483---
baseline (1 extrinsic ents): ['Police have released an image of a man they want to speak to in connection with a sexual assault on a 15-year-old girl in Hertfordshire.']
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']

---XSUM ID 36341722---
baseline (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £200,000.']
fbs_oracle (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £100,000.']

---XSUM ID 36730443---
baseline (2 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in their last-16 tie.']
fbs_oracle (1 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in the quarter-finals.']

---XSUM ID 36932519---
baseline (3 extrinsic ents): ['Two men have been arrested on suspicion

### Summaries messed by FBS Classifier

In [6]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.fbs_classifier_is_non_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(0, 30)


### Corrected by FBS Classifier

In [7]:
df_inspect = df_sums[
    df_sums.baseline_is_non_factual
    & df_sums.fbs_classifier_is_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(2, 30)
---XSUM ID 36540012---
baseline (1 extrinsic ents): ['A collection of photographs taken during the 1970s Rock Against Racism (RAR) movement have gone on display in Leeds.']
fbs_classifier (0 extrinsic ents): ['A collection of photographs taken by a photographer who was involved with the Rock Against Racism (RAR) movement has gone on display.']

---XSUM ID 36730443---
baseline (2 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in their last-16 tie.']
fbs_classifier (1 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in the quarter-finals.']



### Summaries messed up by Corrector

In [8]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "corrector"
    ]
)

1
---XSUM ID 40498102---
baseline (1 extrinsic ents): ["An exhibition in Iran of cartoonists' depictions of Donald Trump has drawn comparisons with Adolf Hitler."]
corrector (1 extrinsic ents): ['An exhibition in Tehran of cartoons of Trump has attracted more than 2,000 entries, many of which have drawn comparisons to Adolf Hitler.']



### Summaries were Oracle > Corrector

In [9]:
df_inspect = df_sums[
    df_sums.fbs_oracle_is_factual
    & df_sums.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "fbs_oracle",
        "corrector"
    ]
)

8
---XSUM ID 26398483---
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']
corrector (1 extrinsic ents): ['Police have released an image of a man they want to speak to in connection with a sexual assault on a 15-year-old girl in Hertfordshire.']

---XSUM ID 36341722---
fbs_oracle (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £100,000.']
corrector (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £200,000.']

---XSUM ID 36932519---
fbs_oracle (3 extrinsic ents): ['Two men have been arrested on suspicion of drugs offences after ecstasy tablets and cocaine worth more than 400,000 euro (£353,176) were seized by police in Dublin.']
corrector (3 extrinsic ents): ['Two men have been arrested on suspicion of drugs offences after ecstasy tablets and cocai